In [1]:
import math
import collections
import pickle
import random
import scipy
import json

from matplotlib import pyplot as plt

import GPy
import numpy as np
import pandas as pd
from tqdm import tqdm

# from Kernel import MixtureViaSumAndProduct, CategoryOverlapKernel
from InitialData_Gen import initialize
from AcquisitionFunctions import EI, PI, UCB, AcquisitionOnSubspace
# from SamplingCategorical import compute_prob_dist_and_draw_hts
from UpdateCategoricalWeight import compute_reward_for_all_cat_variable, update_weights_for_all_cat_var
from optimization import sample_then_minimize

from AskTell import ask_tell

from scipy.optimize import minimize

from typing import Union, Tuple
from paramz.transformations import Logexp

1. C - List of number of categroies corresponding to each categorical variable
2. Nc - Number of categorical variables
3. Nx - Continuous variables
4. h - categorical
5. x - continuous
6. z - [h, x]

7. nDim - Nc + Nx
8. bounds - Lower and Upper bounds of both Catergorical variables and continuous variables

9. n_iter - Number of iterations to run the algorithm
10. initN - Number of intial data points
11. batch_size (b) - Number of experiments to be generated in each iteration

12. acq_type - Acquisition Type ('EI', 'LCB', 'ThompsonSampling')

### 1. Defining parameters of the system

In [2]:
import sys
myString = sys.path[0]
split_list = myString.split("/")
root = ''
for l in split_list[1:-2]:
    root = root +'/'+ l
    
main_file_path = root + '/RBDJ/'
carbon_source_fiepath = root + '/'

In [3]:
initN = 23
Nc = 1
C_list = [19] #

assert Nc == len(C_list)

Nx = 3
nDim = Nc + Nx
Niter = 1
batch_size = 11

bounds = [{'name': 'Carbon_Type', 'type': 'categorical', 'domain': np.arange(0, 19)}, #Carbon Source type
          {'name': 'Conc_Carbon', 'type': 'continuous', 'domain': (0, 1)},
            {'name': 'Gly_OG', 'type': 'continuous', 'domain': (0, 1)},
            {'name': 'Met_Prod', 'type': 'continuous', 'domain': (0, 1)}]

#0 - lb and 1 - ub

data_param = {'C': C_list, 'Nc': Nc, 'Nx': Nx, 'nDim': Nc+Nx, 'bounds': bounds, 'approach_type':'CoCa',
'prob_type': 'UnConstrained', 'initN': initN}

In [4]:
budget = 55
bestUpperBoundEstimate = 2 * budget / 3
gamma_list = [np.sqrt(C * math.log(C / batch_size) / ((math.e - 1) * batch_size * bestUpperBoundEstimate))
                      for C in C_list]

gamma_list = [g if not np.isnan(g) else 1 for g in gamma_list]

Wc_list_init = [np.ones(C) for C in C_list]
Wc_list = Wc_list_init

In [5]:
Model_background = {}
Model_background = {'gamma_list': gamma_list,  'budget':budget,
                 'bestUpperBoundEstimate': bestUpperBoundEstimate, 'Wc_list_init': Wc_list_init,
                   'Wc_list': Wc_list, 'data_param': data_param}
import pickle
with open('0_ModelBackground.pkl', 'wb') as output:
    # Pickle dictionary using protocol 0.
    pickle.dump(Model_background, output)

### 2. Creating the Initial Dataset

In [6]:
Exp_0 , result_0 = initialize(initN, data_param, seed = 37)
pd.DataFrame(Exp_0).to_csv('0_ExperimentalDesign.csv')


(23, 1)


### 3. Perform conversions to compute the volumes save that

In [7]:
Stock_solid = pd.read_excel(carbon_source_fiepath + 'CarbonSourceInfo.xlsx', 'Stocks_solid')
Stock_liquid = pd.read_excel(carbon_source_fiepath + 'CarbonSourceInfo.xlsx', 'Stocks_liquid')

CARBON NAME

In [8]:
Carbon_Names = Stock_solid['Carbon Source'].values.tolist()
Carbon_Names.append(Stock_liquid['Carbon Source'][1])
Carbon_Names.append(Stock_liquid['Carbon Source'][2])

UPPER BOUNDS

In [9]:
Carbon_Ub = [50]*17 #[g/L]
Carbon_Ub.append(10) # Glycerol [mL/L]
Carbon_Ub.append(50) #Ethanol [mL/L]

OG_Gly_Ub = 100 #[mL/L]
Met_Ub = 100 #[mL/L]

STOCK CONCENTRATIONS

In [ ]:
Stock_Conc = Stock_solid['g/mL'].values.tolist()
Stock_Conc.append(Stock_liquid['mL/mL'][1].tolist())
Stock_Conc.append(Stock_liquid['mL/mL'][2].tolist())

OG_Stock_Conc = Stock_liquid['mL/mL'][0].tolist()

In [ ]:
Selected_Carbon = []
Carbon_mL = []
OG_Gly_mL = []
Met_mL = []


for i in range(initN):
    Selected_Carbon.append(Carbon_Names[int(Exp_0[i,0])])
    temp_factor = Carbon_Ub[int(Exp_0[i,0])]/ Stock_Conc[int(Exp_0[i,0])] 
    # [g/L]/[g/mL] * [mL] --> g * mL * mL/[L * g] --> 10^-6 L --> uL 
    temp = Exp_0[i,1] * temp_factor * 3
    Carbon_mL.append(temp)
    # [mL]*[mL/L] --> 10^-6 L --> 1 uL 
    OG_Gly_mL.append(Exp_0[i,2] * OG_Gly_Ub * 3/OG_Stock_Conc)
    Met_mL.append(Exp_0[i,3] * Met_Ub * 3)
    


Experiment_0_3mL = {'Carbon_Type': Selected_Carbon,
               'Conc_Carbon [uL]': Carbon_mL,
               'Gly_OG [uL]': OG_Gly_mL,
               'Met_Prod [uL]': Met_mL}

In [ ]:
pd.DataFrame(Experiment_0_3mL).to_csv('0_ExperimentPlan_mLValue_3mL.csv')